# Ensamble de Regresores
1. Preparación de datos
2. División de datos
3. Predictores individuales
4. Bagging
5. Random Forest
6. Boosting
7. Votación "hard"
8. Votación "soft"
9. Stacking
10. Guardamos el mejor modelo


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# 1. Preparación de datos

In [3]:
data = pd.read_excel(".datos/Emisiones CO2.xlsx",sheet_name=0)
data.info()

NameError: name 'pd' is not defined

In [ ]:
#Correccion de tipo de datos
data['Tipo_vehiculo']=data['Tipo_vehiculo'].astype('category')
data.info()

In [1]:
data['Tipo_vehiculo'].value_counts().plot(kind='bar')

NameError: name 'data' is not defined

In [ ]:
data.plot(kind='box')

In [ ]:
#Normalizacion las variables numéricas (las dummies no se normalizan)
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
variables_numericas=['Tamano_motor','Cilindros','Consumo_combustible_ciudad','Consumo_combustible_carretera']
min_max_scaler.fit(data[variables_numericas]) #Ajuste de los parametros: max - min
data[variables_numericas]= min_max_scaler.transform(data[variables_numericas])
data.head()

In [ ]:
#Dummies para las variables predictoras
data = pd.get_dummies(data, columns=['Tipo_vehiculo'], drop_first=False)
data.head()

# **2.División 70-30**

In [ ]:
#División 70-30
from sklearn.model_selection import train_test_split
X = data.drop("CO2_Emision", axis = 1)
Y = data['CO2_Emision']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3) #En regresion no es muestreo estratificado
Y_train.plot(kind='hist')

# **3. Predictores individuales (base learners)**

In [ ]:
#Dataframe para comparar los resultados
medidas= pd.DataFrame(index=['mse','rmse','mae','mape','max'])

In [ ]:
#Arbol de clasificación
from sklearn.tree import DecisionTreeRegressor
model_dt = DecisionTreeRegressor(criterion='squared_error', min_samples_leaf=20, max_depth=4)
model_dt.fit(X_train, Y_train) # 70%

#Evaluación
from sklearn import metrics
Y_pred = model_dt.predict(X_test)#  30%


mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Arbol']=[mse, rmse, mae, mape,max]
medidas


In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(30,30))
plot_tree(model_dt, feature_names=X_train.columns.values,  rounded=True, filled=True)
plt.show()

In [ ]:
model_dt.feature_names_in_

In [ ]:
model_dt.feature_importances_


In [ ]:
#Método Perezoso
from sklearn.neighbors import KNeighborsRegressor
model_knn = KNeighborsRegressor(n_neighbors=1, metric='euclidean') #minkowski
model_knn.fit(X_train, Y_train)

#Evaluación
from sklearn import metrics
Y_pred = model_knn.predict(X_test)
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Knn']=[mse, rmse, mae, mape,max]
medidas




In [ ]:
#Red neuronal
from sklearn.neural_network import MLPRegressor
model_rn = MLPRegressor(activation="logistic",hidden_layer_sizes=(10), learning_rate='adaptive',
                     learning_rate_init=0.2, momentum= 0.3, max_iter=500, verbose=False)
model_rn.fit(X_train, Y_train)

#Evaluación
Y_pred = model_rn.predict(X_test)
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['NN']=[mse, rmse, mae, mape,max]
medidas




# 4. Bagging

In [ ]:
#Bagging: Knn
from sklearn.ensemble import BaggingRegressor
modelo_base=KNeighborsRegressor(n_neighbors=1, metric='euclidean')
model_bag = BaggingRegressor(modelo_base, n_estimators=10, max_samples=0.6)
model_bag.fit(X_train, Y_train)#70%

#Evaluación
Y_pred = model_bag.predict(X_test) #30%
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Bagging']=[mse, rmse, mae, mape,max]
medidas


# 5. Random Forest (bagging)

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
model_rf= RandomForestRegressor(n_estimators=100,  max_samples=0.7, criterion='squared_error',
                              max_depth=None, min_samples_leaf=2)
model_rf.fit(X_train, Y_train) #70%


#Evaluación
Y_pred = model_rf.predict(X_test) #30%
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Random_Forest']=[mse, rmse, mae, mape,max]
medidas


In [ ]:
# Se imprimen la importancia de las características
print('Importancia de las características')
for i, j in sorted(zip(X_train.columns, model_rf.feature_importances_)):
    print(i, j)

# 6. Boosting

In [ ]:
#Boosting
from sklearn.ensemble import AdaBoostRegressor
modelo_base=DecisionTreeRegressor(criterion='squared_error', max_depth=None, min_samples_leaf=2)
model_boos = AdaBoostRegressor(modelo_base, n_estimators=50)
model_boos.fit(X_train, Y_train)#70%

#Evaluación
Y_pred = model_boos.predict(X_test)#30%
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Boosting']=[mse, rmse, mae, mape,max]
medidas

Librerías y Métodos nuevos de tipo boosting
* GradientBoostingRegressor (sklearn)
* Librería XGBoost
* Librería Catboost (no se crean dummies, trabaja con variables categóricas)

# 7. Votación "hard"

In [ ]:
#Votación hard
from sklearn.ensemble import VotingRegressor

regresores= [('dt', model_dt), ('knn', model_knn), ('net', model_rn)]

model_vot_hard = VotingRegressor(estimators=regresores, weights=None)
model_vot_hard.fit(X_train,Y_train) #70%

#Evaluación
Y_pred = model_vot_hard.predict(X_test)#30%
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Votacion_hard']=[mse, rmse, mae, mape,max]
medidas


# 8. Votación "soft"

In [ ]:
#Votación soft

from sklearn.ensemble import VotingRegressor

regresores= [('dt', model_dt), ('knn', model_knn), ('net', model_rn)]
model_vot_soft = VotingRegressor(estimators=regresores, weights=[0.3, 0.4, 0.3])
model_vot_soft.fit(X_train,Y_train) #70%

#Evaluación
Y_pred = model_vot_soft.predict(X_test) #30%
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Votacion_soft']=[mse, rmse, mae, mape,max]
medidas



# 9. Stacking

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

regresores= [('dt', model_dt), ('knn', model_knn), ('net', model_rn)]

metodo_ensamblador= LinearRegression()

#metodo_ensamblador = SVR(kernel='linear', probability=True) #'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'

model_stack= StackingRegressor(estimators=regresores, final_estimator=metodo_ensamblador)
model_stack.fit(X_train,Y_train)#70%

#Evaluación
Y_pred = model_stack.predict(X_test) #30%
mse = metrics.mean_squared_error(Y_test,Y_pred) # Entre mas pequeño mejor
rmse = np.sqrt(mse) #diferencia entre el valor real y la prediccion
mae= metrics.mean_absolute_error(Y_test,Y_pred) #diferencia entre el valor real y la prediccion
mape=metrics.mean_absolute_percentage_error(Y_test,Y_pred) # error en porcentaje
max=metrics.max_error(Y_test,Y_pred)
medidas['Stacking']=[mse, rmse, mae, mape,max]
medidas


# **10. Guardamos el mejor modelo**

In [ ]:
import pickle
filename = 'modelo-ensamble-reg.pkl'
variables=X.columns._values
pickle.dump([model_boos,variables, min_max_scaler], open(filename, 'wb'))

